In [2]:
import wandb

In [3]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install setuptools==58

  Obtaining dependency information for setuptools==58 from https://files.pythonhosted.org/packages/57/5a/80fd2846d0151aa81a13540afc97be49f8fa19752933172b76854ba90adc/setuptools-58.0.0-py3-none-any.whl.metadata
Using cached setuptools-58.0.0-py3-none-any.whl (816 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.0.0
    Uninstalling setuptools-68.0.0:
      Successfully uninstalled setuptools-68.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
spyder 5.4.3 requires pyqt5<5.16, which is not installed.
spyder 5.4.3 requires pyqtwebengine<5.16, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0 which is incompatible.
conda

In [5]:
pip install pygooglenews --upgrade

  Obtaining dependency information for pygooglenews from https://files.pythonhosted.org/packages/59/cb/84f162888a07e501630f6be7c70997c5c8afd5ba4a40c3a079c321e97c29/pygooglenews-0.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for dateparser<0.8.0,>=0.7.6 from https://files.pythonhosted.org/packages/c1/d5/5a2e51bc0058f66b54669735f739d27afc3eb453ab00520623c7ab168e22/dateparser-0.7.6-py2.py3-none-any.whl.metadata
  Using cached feedparser-5.2.1.zip (1.2 MB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tzlocal from https://files.pythonhosted.org/packages/97/3f/c4c51c55ff8487f2e6d0e618dba917e3c3ee2caae6cf0fbb59c9b1876f2e/tzlocal-5.2-py3-none-any.whl.metadata
Using cached pygooglenews-0.1.2-py3-none-any.whl (10 kB)
Using cached dateparser-0.7.6-py2.py3-none-any.whl (362 kB)
Using cached tzlocal-5.2-py3-none-any.whl (17 kB)
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=45011 sha256=c452f2f2b71c772514a14b412

In [6]:
pip install --upgrade feedparser

  Obtaining dependency information for feedparser from https://files.pythonhosted.org/packages/7c/d4/8c31aad9cc18f451c49f7f9cfb5799dadffc88177f7917bc90a66459b1d7/feedparser-6.0.11-py3-none-any.whl.metadata
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=cc30f52e448d808e69cf2049cfc3d5e359161e842d0b334b7a415ff48769846d
  Stored in directory: /Users/nikitasomani/Library/Caches/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Attempting uninstall: feedparser
    Found existing installation: feedparser 5.2.1
    Uninstalling feedparser-5.2.1:
      Successfully uninstalled feedparser-5.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [7]:
from pygooglenews import GoogleNews
gn = GoogleNews()

In [8]:
headline_list= []
crypto_list = []

CRYPTOs = ['bitcoin' ,'eth', 'cardano', 'safemoon', 'altcoin', 'litecoin', 'fantom' ]
for crypto in CRYPTOs:
    response = gn. search (crypto)
    for r in response['entries']:
        headline_list.append(r['title'])
        crypto_list.append (crypto)

In [9]:
len(headline_list)

695

In [10]:
# pip install transformers

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained ( "ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/nikitasomani/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [12]:
import wandb
wandb.init(project="CryptoCurrency Headline_Sentiment_Analysis")
headlines_table = wandb.Table(columns=["Headline", "Crypto", "Positive", "Negative", "Neutral"] )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: niksom406 (sentiment_na). Use `wandb login --relogin` to force relogin


In [13]:
import torch
def chunkList(list,n) :
    for i in range(0, len(list),n):
        yield list[i:i+n]
n=0
chunkSize = 100
model.eval()
for lines, cryptos in zip(chunkList (headline_list,chunkSize), chunkList (crypto_list, chunkSize) ):
    input = tokenizer (lines, padding=True, truncation=True, return_tensors='pt')
    outputs = model (**input)
    prediction = torch.nn.functional.softmax(outputs.logits,dim=-1)
    print(f" {n+1}/{int(len (headline_list)/chunkSize)}")
    for headline, crypto, pos, neg, neutr in zip(lines, cryptos, prediction[:,0].tolist(), prediction[:, 1]. tolist (), prediction[:, 1]. tolist ()):
        headlines_table.add_data (headline,crypto,pos,neg, neutr)
        n=n+1
print(type(headlines_table))

 1/6
 101/6
 201/6
 301/6
 401/6
 501/6
 601/6
<class 'wandb.data_types.Table'>


In [14]:
wandb.run.log({"Cryptocurrency Sentiment Analysis Table ": headlines_table})
wandb.run.finish()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [22]:
import pandas as pd
table_data = headlines_table.data
df = pd.DataFrame(table_data, columns=headlines_table.columns)
df.to_csv("/Users/nikitasomani/Desktop/Hackathon/AIHackathon24/AI_HFT_Quantum_Project/data/processed/Sentimental_analysis.csv", index=False)

In [21]:
df.head(10)

,Headline,Crypto,Positive,Negative,Neutral
0,"Bitcoin Halving: What It Means, Potential Impa...",bitcoin,0.032197,0.264307,0.264307
1,Bitcoin Miners Are Devouring Energy at a Recor...,bitcoin,0.056784,0.043786,0.043786
2,How to capitalize on the Bitcoin halving - Coi...,bitcoin,0.118363,0.011485,0.011485
3,"Bitcoin breaks $70,000 in volatile trading, hi...",bitcoin,0.512939,0.071788,0.071788
4,DeFi TVL reaches $100B as Bitcoin pumps sentim...,bitcoin,0.876630,0.016265,0.016265
5,Bitcoin’s black swan event: How the return of ...,bitcoin,0.055200,0.043232,0.043232
6,SEC's Gensler says crypto 'rife with abuses an...,bitcoin,0.440768,0.169024,0.169024
7,‘More Dangerous’ Than Nukes—China And Russia R...,bitcoin,0.094116,0.222146,0.222146
8,Why Bitcoin Just Hit Its All - TIME,bitcoin,0.245069,0.034447,0.034447
9,"This Week in Coins: Bitcoin Touches $70,000 As...",bitcoin,0.445889,0.074236,0.074236
